# Setup Minedojo Environment

In [1]:
from Environments import SkyRunner, MultithreadGym
frame_stack = True
frames = 3

# Multithreaded environment wrapper
env = MultithreadGym.MultithreadGym(thread_int=1, env_int=1, frame_stack=frame_stack, frames_int=frames)

# Evaluation environment
eval_env = SkyRunner.CustomEnv(frame_stack=frame_stack, frames_int=frames)

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


starting Reloader 0
ThreadID 0 waiting for environment to reset.
ThreadID: 0 has received an enviornment from queue. Reset of envornment is being prepeared


# Begin training

In [2]:
import train_openAI
import importlib

importlib.reload(train_openAI)

train_openAI.train(env, eval_env=eval_env, eval_freq=5000)

Waiting for ready enviornment


[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


ThreadID 0 put complete environment into ready-queue.Ready enviornment received

ThreadID 0 waiting for environment to reset.
Inventory and weather cleared!
Inventory and weather cleared!Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.

Inventory and weather cleared!
Inventory and weather cleared!
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Inventory and weather cleared!
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Inventory and weather cleared!
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Broken block detected. Moving to location 1
Inventory and weather cleared!
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Inventory and weather cleared!
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environment to reset.
Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
ThreadID 0 waiting for environ

KeyboardInterrupt: 

# Shutdown Environments

In [3]:
env.close()

Queue-GET timed out. Trying again, if not exit_flag has been sat. ThreadID: 0
stopping Reloader 0


# Load existing model

In [ ]:
from stable_baselines3 import DQN
from CustomBaselines3.DoubleDQN import DoubleDQN

_env = SkyRunner.CustomEnv()
model = DQN.load("last_model_working.zip")

obs = env.reset()
acc_r = 0
while True:
    act, st = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(act)

    acc_r += reward

    env.render()

    if done:
        obs = env.reset()
        print("Finished with reward %d" % acc_r)
        acc_r = 0

Finished with reward 7
Finished with reward 18
Finished with reward 0
Finished with reward 0
Finished with reward 10
Finished with reward 8
Finished with reward 19
Finished with reward 0
Finished with reward 0
Finished with reward 19
Finished with reward 11
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 4
Finished with reward 0
Finished with reward 8
Finished with reward 1004
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 1017
Finished with reward 1004
Finished with reward 1017
Finished with reward 13
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 14
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 0
Finished with reward 7
Finished with reward 1017
Finished with reward 0
Finished with reward 0
Finished with reward 19
Finished with reward 0
Finished with reward 14
Finished w